In [1]:
%load_ext autoreload
from google.auth import default
credentials, project = default()

In [ ]:

%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.config_loader import config
from src.shop import generate_product_images, generate_shopify_csv
from src.zip import create_m4a_zip_collections
config.TARGET_LANGUAGE_NAME

In [4]:

BUNDLE_CONFIG = {
            "Bundle 03-04": [3, 4],
            #"Bundle 09-14": [9, 14],
            #"Bundle 15-20": [15, 20],
        }

PRICES = {"individual": 2.99, "bundle": 5.99, "complete": 19.99}



In [13]:
create_m4a_zip_collections(bundle_config=BUNDLE_CONFIG,collection="LM1000")

Creating M4A zip collections for LM1000 in French
Found 20 stories total

=== Creating Complete Collection Zip ===

Creating Complete French LM1000 Audio Collection (120 files)
Files to be included:
  - french_01_story_sunset_wedding_blues_introduction.m4a (from ..\outputs\gcs\audio-language-trainer-private-content\collections\LM1000\french\audio\french_01_story_sunset_wedding_blues_introduction.m4a)
  - french_01_story_sunset_wedding_blues_introduction_FAST.m4a (from ..\outputs\gcs\audio-language-trainer-private-content\collections\LM1000\french\audio\french_01_story_sunset_wedding_blues_introduction_FAST.m4a)
  - french_01_story_sunset_wedding_blues_development.m4a (from ..\outputs\gcs\audio-language-trainer-private-content\collections\LM1000\french\audio\french_01_story_sunset_wedding_blues_development.m4a)
  - french_01_story_sunset_wedding_blues_development_FAST.m4a (from ..\outputs\gcs\audio-language-trainer-private-content\collections\LM1000\french\audio\french_01_story_sunset_w

Adding files:   0%|          | 0/120 [00:00<?, ?it/s]

Adding french_01_story_sunset_wedding_blues_introduction.m4a (472.0 KB)
Adding french_01_story_sunset_wedding_blues_introduction_FAST.m4a (1219.5 KB)


Adding files:   2%|▎         | 3/120 [00:00<00:04, 27.27it/s]

Adding french_01_story_sunset_wedding_blues_development.m4a (278.2 KB)
Adding french_01_story_sunset_wedding_blues_development_FAST.m4a (1437.0 KB)
Adding french_01_story_sunset_wedding_blues_resolution.m4a (225.6 KB)
Adding french_01_story_sunset_wedding_blues_resolution_FAST.m4a (1169.9 KB)


...


Zip file created in memory: 6.4 MB

Writing zip file to local path: ..\outputs\gcs\audio-language-trainer-private-content\collections\LM1000\french\audio\french_lm1000_story_19_the-birthday-party_audio.zip
Local file written successfully: 6.4 MB

Creating French LM1000 Story 20: The Power Cut Audio (6 files)
Files to be included:
  - french_20_story_the_power_cut_introduction.m4a (from ..\outputs\gcs\audio-language-trainer-private-content\collections\LM1000\french\audio\french_20_story_the_power_cut_introduction.m4a)
  - french_20_story_the_power_cut_introduction_FAST.m4a (from ..\outputs\gcs\audio-language-trainer-private-content\collections\LM1000\french\audio\french_20_story_the_power_cut_introduction_FAST.m4a)
  - french_20_story_the_power_cut_development.m4a (from ..\outputs\gcs\audio-language-trainer-private-content\collections\LM1000\french\audio\french_20_story_the_power_cut_development.m4a)
  - french_20_story_the_power_cut_development_FAST.m4a (from ..\outputs\gcs\audio-lang

Adding french_20_story_the_power_cut_introduction.m4a (279.3 KB)
Adding french_20_story_the_power_cut_introduction_FAST.m4a (1445.4 KB)
Adding french_20_story_the_power_cut_development.m4a (327.5 KB)
Adding french_20_story_the_power_cut_development_FAST.m4a (1679.0 KB)


Creating individual zips: 100%|██████████| 20/20 [00:07<00:00,  2.81it/s]

Adding french_20_story_the_power_cut_resolution.m4a (334.8 KB)
Adding french_20_story_the_power_cut_resolution_FAST.m4a (1705.8 KB)

Zip file created in memory: 5.6 MB

Writing zip file to local path: ..\outputs\gcs\audio-language-trainer-private-content\collections\LM1000\french\audio\french_lm1000_story_20_the-power-cut_audio.zip
Local file written successfully: 5.6 MB

🎉 Created 22 zip files successfully!
Zip files saved to: ..\outputs\gcs\audio-language-trainer-private-content\collections\LM1000\french\audio


{'complete': '..\\outputs\\gcs\\audio-language-trainer-private-content\\collections\\LM1000\\french\\audio\\french_lm1000_complete_audio_collection.zip',
 'bundle_03_04': '..\\outputs\\gcs\\audio-language-trainer-private-content\\collections\\LM1000\\french\\audio\\french_lm1000_bundle_03_04_audio.zip',
 'individual_story_sunset_wedding_blues': '..\\outputs\\gcs\\audio-language-trainer-private-content\\collections\\LM1000\\french\\audio\\french_lm1000_story_01_sunset-wedding-blues_audio.zip',
 'individual_story_better_than_a_movie': '..\\outputs\\gcs\\audio-language-trainer-private-content\\collections\\LM1000\\french\\audio\\french_lm1000_story_02_better-than-a-movie_audio.zip',
 'individual_story_unexpected_wedding_guests': '..\\outputs\\gcs\\audio-language-trainer-private-content\\collections\\LM1000\\french\\audio\\french_lm1000_story_03_unexpected-wedding-guests_audio.zip',
 'individual_story_midnight_garden_mystery': '..\\outputs\\gcs\\audio-language-trainer-private-content\\coll

In [ ]:
# Default configuration (stories 1-8, 9-14, 15-20)
print("=== Generating Product Images ===")
generated_images = generate_product_images(bundle_config=BUNDLE_CONFIG,
    collection="LM1000", generate_individual=False
)

print("\nGenerated images:")
for product_type, uri in generated_images.items():
    print(f"  {product_type}: {uri}")


In [ ]:

# 2. Generate the Shopify CSV with default settings
print("\n=== Generating Shopify CSV ===")

csv_path = generate_shopify_csv(
    bundle_config=BUNDLE_CONFIG,
    prices=PRICES,
    collection="LM1000",
    free_individual_count=2
)

print(f"CSV generated at: {csv_path}")

